In [5]:
import pandas as pd
import numpy as np
from decimal import Decimal as Dec
import matplotlib.pyplot as plt
import sympy
import os

pd.options.mode.chained_assignment = None  # default='wsarn'
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

# numerical results

In [100]:
folder_path = "numerical_detail_a_0_1"
num_results_df = pd.DataFrame()

for csv in sorted(os.listdir(folder_path)):
    row_dict = {}
    if "epoc" in csv:
        df = pd.read_csv(folder_path + "/" + csv)
        df = df.iloc[:-1]
        row_dict['pool'] = csv
        b = float(csv[:-9].replace("_", "."))
        row_dict['b'] = b
        row_dict['rebase_time'] = len(df) - 1
        row_dict['swap_time'] = df.iloc[-1]['epoc_end_index'] - 1
        row_dict['mean_stopping_time'] = row_dict['swap_time'] / row_dict['rebase_time']

        row_dict['pnl_mean'] = np.mean(df['end_wealth'] - df['invest_wealth'])
        row_dict['pnl_IL_mean'] = np.mean(df['end_wealth'] - df['holding_wealth'])
        row_dict['fee_wealth_mean'] = np.mean(df['end_wealth_fee'])
        row_dict['IL_mean'] = np.mean(df['end_wealth_in_pool'] - df['holding_wealth'])
        # row_dict['theo_IL'] = -(np.exp(b) - np.exp(-b)) / 4 * 20
        # row_dict['theo_pnl_mean'] = row_dict['theo_IL'] + row_dict['fee_wealth_mean']

        row_dict['pnl_apr'] = row_dict['pnl_mean'] / np.mean(df['invest_wealth'])
        row_dict['pnl_IL_apr'] = row_dict['pnl_IL_mean'] / np.mean(df['invest_wealth'])
        row_dict['fee_wealth_apr'] = row_dict['fee_wealth_mean'] / np.mean(df['invest_wealth'])
        row_dict['IL_mean_apr'] = row_dict['IL_mean'] / np.mean(df['invest_wealth'])
        # row_dict['theo_IL_apr'] = row_dict['theo_IL'] / np.mean(df['invest_wealth'])
        # row_dict['theo_pnl_apr'] = row_dict['theo_pnl_mean'] / np.mean(df['invest_wealth'])
        row_dict['effective_tx_ratio'] = sum(df['epoc_effective_tx_count']) / row_dict['swap_time']

        num_results_df = num_results_df.append(row_dict, ignore_index=True)

demo_df = num_results_df[['pool', 'b', 'rebase_time', 'swap_time', 'mean_stopping_time', 'fee_wealth_mean',
                          'IL_mean', 'pnl_mean', 'pnl_IL_mean', 'fee_wealth_apr', 'IL_mean_apr', 
                          'pnl_apr', 'pnl_IL_apr', 'effective_tx_ratio']]
demo_df
demo_df.to_csv("numerical_detail_a_0_1.csv")

In [111]:
source_folder = "pool_detail"
summary_folder = "pool_detail_summary"

for pool in sorted(os.listdir(source_folder)):
    if "." not in pool:
        num_results_df = pd.DataFrame()
        for csv in sorted(os.listdir(source_folder + "/" + pool)):
            row_dict = {}
            if "epoc" in csv:
                df = pd.read_csv(source_folder + "/" + pool + "/" + csv)
                # df = df.iloc[:-1]
                row_dict['pool'] = pool
                if "optimum" not in csv:
                    b = float(csv[:-9].replace("_", "."))
                    row_dict['b'] = b
                else:
                    row_dict['b'] = "optimum"
                row_dict['rebase_time'] = len(df) - 1
                row_dict['swap_time'] = df.iloc[-1]['epoc_end_index'] - 1
                row_dict['mean_stopping_time'] = row_dict['swap_time'] / row_dict['rebase_time']
                row_dict['pnl_mean'] = np.mean(df['end_wealth'] - df['invest_wealth'])
                row_dict['pnl_IL_mean'] = np.mean(df['end_wealth'] - df['holding_wealth'])
                row_dict['fee_wealth_mean'] = np.mean(df['end_wealth_fee'])
                row_dict['IL_mean'] = np.mean(df['end_wealth_in_pool'] - df['holding_wealth'])
                row_dict['theo_IL'] = -(np.exp(b) - np.exp(-b)) / 4 * 20
                row_dict['theo_pnl_mean'] = row_dict['theo_IL'] + row_dict['fee_wealth_mean']

                row_dict['pnl_apr'] = row_dict['pnl_mean'] / np.mean(df['invest_wealth'])
                row_dict['pnl_IL_apr'] = row_dict['pnl_IL_mean'] / np.mean(df['invest_wealth'])
                row_dict['fee_wealth_apr'] = row_dict['fee_wealth_mean'] / np.mean(df['invest_wealth'])
                row_dict['IL_mean_apr'] = row_dict['IL_mean'] / np.mean(df['invest_wealth'])
                row_dict['theo_IL_apr'] = row_dict['theo_IL'] / np.mean(df['invest_wealth'])
                row_dict['theo_pnl_apr'] = row_dict['theo_pnl_mean'] / np.mean(df['invest_wealth'])
                # row_dict['effective_tx_ratio'] = sum(df['epoc_effective_tx_count']) / row_dict['swap_time']

                num_results_df = num_results_df.append(row_dict, ignore_index=True)
            
            demo_df = num_results_df[['pool', 'b', 'rebase_time', 'swap_time', 'mean_stopping_time', 'fee_wealth_mean',
                          'IL_mean', 'theo_IL', 'pnl_mean', 'pnl_IL_mean', 'theo_pnl_mean', 'fee_wealth_apr', 'IL_mean_apr', 
                          'theo_IL_apr', 'pnl_apr', 'pnl_IL_apr', 'theo_pnl_apr']]

            demo_df.to_csv(summary_folder + "/" + pool + "_summary.csv")

<ipython-input-111-9245783b7819>:20: RuntimeWarning: divide by zero encountered in long_scalars
  row_dict['mean_stopping_time'] = row_dict['swap_time'] / row_dict['rebase_time']
<ipython-input-111-9245783b7819>:20: RuntimeWarning: divide by zero encountered in double_scalars
  row_dict['mean_stopping_time'] = row_dict['swap_time'] / row_dict['rebase_time']


In [85]:
source_folders = "pool_detail"
results_df = pd.DataFrame()
for pool in sorted(os.listdir(source_folders)):
    try:
        row_dict = {}
        folder_path = source_folders + "/" + pool
        profit_dict = {}

        row_dict['pool'] = pool
        for csv in sorted(os.listdir(folder_path)):
            if "epoc" in csv and "optimum" not in csv:
                df = pd.read_csv(folder_path + "/" + csv)
                profit_dict[csv] = sum(df['end_wealth'] - df['holding_wealth']) / max(df['invest_wealth'])
        max_pnl = max(profit_dict.values())
        for (k, v) in profit_dict.items():
            if v == max_pnl:
                row_dict['best_b'] = k
                row_dict['best_b_apr'] = v
                row_dict['b'] = float(k[:-9].replace("_", "."))
                # row_dict['theo_IL'] = -(np.exp(row_dict['b']) - np.exp(-row_dict['b'])) / 4 * 200 / max(df['invest_wealth'])
                # row_dict['best_IL'] = sum((df['holding_wealth'] - df['end_wealth']) * (df['epoc_end_index'] - df['epoc_begin_index'])) / df.iloc[-1]['epoc_end_index']
                row_dict['best_fee_apr'] = sum(df['end_wealth_fee']) / max(df['invest_wealth'])
                # row_dict['best_IL_apr'] = sum(df['end_wealth_in_pool'] - df['holding_wealth']) / max(df['invest_wealth'])
                break

        csv = "optimum_epoc.csv"
        df = pd.read_csv(folder_path + "/" + csv)
        row_dict['optimum_b'] = csv
        row_dict['optimum_b_apr'] = sum(df['end_wealth'] - df['holding_wealth']) / max(df['invest_wealth'])
        # row_dict['optimum_IL'] = sum((df['holding_wealth'] - df['end_wealth']) * (df['epoc_end_index'] - df['epoc_begin_index'])) / df.iloc[-1]['epoc_end_index']
        row_dict['optimum_fee_apr'] = sum(df['end_wealth_fee']) / max(df['invest_wealth'])
        # row_dict['optimum_IL_apr'] = sum(df['end_wealth_in_pool'] - df['holding_wealth']) / max(df['invest_wealth'])
        results_df = results_df.append(row_dict, ignore_index=True)
    except Exception as e: pass

In [86]:
demo_df = results_df[['pool', 'best_b', 'best_b_apr', 'best_fee_apr', 'best_IL_apr', 'optimum_b_apr', 'optimum_fee_apr', 'optimum_IL_apr']]
demo_df.to_csv("pool_results.csv")

# get price data

In [16]:
des_path = "pool_price_hist"
for folder_name in sorted(os.listdir("pool_detail_backup")):
    if "." not in folder_name:
        df = pd.read_csv("pool_detail_backup/" + folder_name + "/0_2_transaction.csv")
        df = df[['swap_index', 'current_price']]
        df.to_csv(des_path + "/" + folder_name + "_price_hist.csv", index=False)


In [107]:
df = pd.read_csv("after_preprocessing/AAVE_WETH_3000_0815.csv")
df['timeStamp']

0       2021-05-05 17:43:44
1       2021-05-05 17:43:44
2       2021-05-05 20:29:05
3       2021-05-05 20:34:00
4       2021-05-05 20:34:00
               ...         
8246    2021-08-12 13:55:56
8247    2021-08-12 13:59:34
8248    2021-08-12 14:11:44
8249    2021-08-12 14:16:29
8250    2021-08-12 14:26:25
Name: timeStamp, Length: 8251, dtype: object